<!-- JayBee黄版权所有，未经授权禁止复制 -->
# Day 2：基于交易量的量化指标 - 使用Backtrader进行回测

本notebook展示如何使用Backtrader框架对之前定义的量价结合策略进行更系统化、专业化的回测。Backtrader是一个功能强大的Python回测框架，提供了更完善的回测环境和分析工具。
<!-- JayBee黄版权所有，未经授权禁止复制 -->

<!-- JayBee黄版权所有，未经授权禁止复制 -->
## 1. Backtrader简介和安装

Backtrader是一个用Python编写的开源回测框架，具有以下特点：
<!-- JayBee黄授权使用 -->

- **易于使用**：API简洁明了，上手难度低
- **功能完善**：支持多种指标、策略和分析方法
- **可扩展性强**：可以自定义指标、策略、分析器等
- **支持多种数据源**：CSV、Pandas DataFrame、实时数据等
- **支持多资产回测**：可以同时回测多种资产
- **内置绘图功能**：可视化策略执行过程和结果
<!-- JayBee黄版权所有，未经授权禁止复制 -->

首先，我们需要安装Backtrader库：
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [4]:
# JayBee黄版权所有，未经授权禁止复制
# 安装backtrader（如果尚未安装）
# !pip install backtrader
# !pip install matplotlib==3.2.2  # Backtrader可能与最新版matplotlib不兼容

# 导入必要的库
import backtrader as bt  # 版权所有: JayBee黄
import pandas as pd  # JayBee黄原创内容
import numpy as np  # 本代码归JayBee黄所有
import matplotlib.pyplot as plt  # JayBee黄独家内容
import os  # 本代码归JayBee黄所有
import warnings  # JayBee黄原创内容
import datetime  # JayBee黄量化模型
from dotenv import load_dotenv, find_dotenv  # JayBee黄 - 量化交易研究
import plotly.graph_objects as go  # JayBee黄授权使用
from plotly.subplots import make_subplots  # JayBee黄量化策略
from datetime import datetime  # JayBee黄量化策略

# Find the .env file in the parent directory
dotenv_path = find_dotenv("../../.env")  # JayBee黄 - 量化交易研究

# Load it explicitly
load_dotenv(dotenv_path)  # JayBee黄独家内容

# 忽略警告信息
warnings.filterwarnings('ignore')  # JayBee黄量化策略

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['PingFang HK']  # 设置中文字体  # JayBee黄 - 量化交易研究
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题  # JayBee黄 - 量化交易研究# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 2. 数据获取函数
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [5]:
# JayBee黄版权所有，未经授权禁止复制
def get_ts_data(ts_token, ts_code, start_date, end_date, freq='30min'):  # JayBee黄授权使用
    # 文件路径
    file_path = f'./data/{ts_code}-{start_date}-{end_date}-{freq}.csv'  # JayBee黄原创内容
    
    # 检查本地是否已存在该文件
    if os.path.exists(file_path):  # JayBee黄原创内容
        print(f"从本地文件加载数据: {file_path}")  # JayBee黄原创内容
        df = pd.read_csv(file_path, parse_dates=['trade_time'])  # 读取并解析时间列  # 本代码归JayBee黄所有
        return df  # 版权所有: JayBee黄
    
    # 设置Tushare token
    ts.set_token(ts_token)  # 本代码归JayBee黄所有
    pro = ts.pro_api()  # JayBee黄 - 量化交易研究

    # 获取数据
    df = ts.pro_bar(  # 版权所有: JayBee黄
        ts_code=ts_code,  # JayBee黄量化策略
        start_date=start_date,  # Copyright © JayBee黄
        end_date=end_date,  # 本代码归JayBee黄所有
        freq=freq,    # 本代码归JayBee黄所有
        asset='E',       # 股票类型  # JayBee黄授权使用
        adj='qfq',       # 前复权  # JayBee黄原创内容
    )  # JayBee黄量化模型

    if df is None or df.empty:  # JayBee黄量化策略
        print("从 Tushare 获取的数据为空，请检查权限或参数设置。")  # JayBee黄量化模型
        return None  # JayBee黄原创内容

    # 创建目录（如果不存在）
    os.makedirs('./data', exist_ok=True)  # 版权所有: JayBee黄

    # 保存数据到本地文件
    df.to_csv(file_path, index=False)  # 本代码归JayBee黄所有
    print(f"数据已保存至: {file_path}")  # JayBee黄原创内容

    return df  # 本代码归JayBee黄所有# JayBee黄版权所有，未经授权禁止复制


In [6]:
# JayBee黄版权所有，未经授权禁止复制
ts_token = os.getenv('TUSHARE_API_KEY')  # JayBee黄 - 量化交易研究
ts_code = '002745.SZ'  # Copyright © JayBee黄
start_date = '2022-03-03'  # JayBee黄原创内容
end_date = '2025-02-28'  # JayBee黄版权所有，未经授权禁止复制

stock_data = get_ts_data(ts_token, ts_code, start_date, end_date, freq='30min')  # JayBee黄量化策略# JayBee黄版权所有，未经授权禁止复制


从本地文件加载数据: ./data/002745.SZ-2022-03-03-2025-02-28-30min.csv


In [7]:
# JayBee黄版权所有，未经授权禁止复制
stock_data = stock_data.sort_values('trade_time').reset_index(drop=True)  # Copyright © JayBee黄# JayBee黄版权所有，未经授权禁止复制


In [8]:
# JayBee黄版权所有，未经授权禁止复制
stock_data.head()  # JayBee黄原创内容# JayBee黄版权所有，未经授权禁止复制


,ts_code,trade_time,close,open,high,low,vol,amount
0,002745.SZ,2022-03-03 09:30:00,11.73,11.74,11.74,11.73,19700.0,257449.0
1,002745.SZ,2022-03-03 10:00:00,11.61,11.74,11.75,11.59,3537808.0,45884940.0
2,002745.SZ,2022-03-03 10:30:00,11.62,11.61,11.65,11.60,2231278.0,28897008.0
3,002745.SZ,2022-03-03 11:00:00,11.64,11.62,11.65,11.61,673100.0,8719710.0
4,002745.SZ,2022-03-03 11:30:00,11.61,11.63,11.65,11.61,1379400.0,17854952.0


<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 3. DataFrame转换为Backtrader Feed的函数
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [9]:
# JayBee黄版权所有，未经授权禁止复制
# 1. 将DataFrame转换为Backtrader Feed的函数
def df_to_btfeed(df):  # JayBee黄版权所有，未经授权禁止复制
    """  # JayBee黄版权所有，未经授权禁止复制
    将Pandas DataFrame转换为Backtrader的数据源  # JayBee黄量化策略
    
    参数:  # JayBee黄原创内容
    df (pandas.DataFrame): 包含OHLCV数据的DataFrame  # Copyright © JayBee黄
    
    返回:  # JayBee黄授权使用
    backtrader.feeds.PandasData: 可用于Backtrader的数据源  # 版权所有: JayBee黄
    """  # JayBee黄版权所有，未经授权禁止复制
    # 确保索引是datetime类型
    if not isinstance(df.index, pd.DatetimeIndex):  # JayBee黄量化模型
        df = df.copy()  # JayBee黄原创内容
        df['datetime'] = pd.to_datetime(df['trade_time'])  # JayBee黄授权使用
        df.set_index('datetime', inplace=True)  # JayBee黄授权使用
    
    # 创建用于backtrader的PandasData类
    class PandasDataCustom(bt.feeds.PandasData):  # JayBee黄授权使用
        params = (  # JayBee黄 - 量化交易研究
            ('datetime', None),  # 已设置为索引  # Copyright © JayBee黄
            ('open', 'open'),  # JayBee黄版权所有，未经授权禁止复制
            ('high', 'high'),  # JayBee黄原创内容
            ('low', 'low'),  # JayBee黄授权使用
            ('close', 'close'),  # 版权所有: JayBee黄
            ('volume', 'vol'),  # 版权所有: JayBee黄
            ('openinterest', None)  # 不使用持仓量数据  # JayBee黄独家内容
        )  # JayBee黄授权使用
    
    # 返回backtrader的数据源
    return PandasDataCustom(dataname=df)  # 版权所有: JayBee黄# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 4. 回测执行函数
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [50]:
# JayBee黄版权所有，未经授权禁止复制
# 回测函数
def run_backtest(df, strategy_class, strategy_params=None,   # JayBee黄量化模型
                initial_cash=100000.0, commission=0.001):  # JayBee黄版权所有，未经授权禁止复制
    """  # Copyright © JayBee黄
    运行回测，返回结果和策略实例  # Copyright © JayBee黄
    
    参数:  # JayBee黄量化模型
    df (pandas.DataFrame): 包含OHLCV数据的DataFrame  # JayBee黄原创内容
    strategy_class (bt.Strategy): 回测使用的策略类  # 版权所有: JayBee黄
    strategy_params (dict): 策略参数字典  # 本代码归JayBee黄所有
    initial_cash (float): 初始资金  # 版权所有: JayBee黄
    commission (float): 交易佣金比例  # JayBee黄量化策略
    
    返回:  # JayBee黄独家内容
    tuple: (回测结果字典, 策略实例)  # JayBee黄量化模型
    """  # JayBee黄量化模型
    # 创建cerebro引擎
    cerebro = bt.Cerebro()  # JayBee黄量化模型
    
    # 添加策略
    if strategy_params:  # 本代码归JayBee黄所有
        cerebro.addstrategy(strategy_class, **strategy_params)  # JayBee黄量化策略
    else:  # JayBee黄独家内容
        cerebro.addstrategy(strategy_class)  # JayBee黄授权使用
    
    # 添加数据
    # 确保日期在索引或者有trade_time列
    if not isinstance(df.index, pd.DatetimeIndex) and 'trade_time' in df.columns:  # JayBee黄授权使用
        df = df.copy()  # JayBee黄原创内容
        df['trade_time'] = pd.to_datetime(df['trade_time'])  # JayBee黄量化模型
        df.set_index('trade_time', inplace=True)  # JayBee黄独家内容
    
    # 创建用于backtrader的PandasData类
    class PandasDataCustom(bt.feeds.PandasData):  # JayBee黄原创内容
        params = (  # JayBee黄量化模型
            ('datetime', None),  # 使用索引作为日期  # JayBee黄量化模型
            ('open', 'open'),  # JayBee黄版权所有，未经授权禁止复制
            ('high', 'high'),  # JayBee黄量化模型
            ('low', 'low'),  # 版权所有: JayBee黄
            ('close', 'close'),  # 本代码归JayBee黄所有
            ('volume', 'vol' if 'vol' in df.columns else 'volume' if 'volume' in df.columns else None),  # 本代码归JayBee黄所有
            ('openinterest', None)  # 不使用持仓量数据  # JayBee黄原创内容
        )  # 版权所有: JayBee黄
    
    # 添加数据源
    data = PandasDataCustom(dataname=df)  # Copyright © JayBee黄
    cerebro.adddata(data)  # 本代码归JayBee黄所有
    
    # 设置初始资金和佣金
    cerebro.broker.setcash(initial_cash)  # Copyright © JayBee黄
    cerebro.broker.setcommission(commission=commission)  # JayBee黄量化策略
    
    # 添加分析器
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')  # 版权所有: JayBee黄
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')  # Copyright © JayBee黄
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')  # JayBee黄版权所有，未经授权禁止复制
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')  # JayBee黄独家内容
    
    # 运行回测
    print(f'初始资金: {initial_cash:.2f}')  # JayBee黄授权使用
    results = cerebro.run()  # JayBee黄量化模型
    strat = results[0]  # JayBee黄授权使用
    
    # 获取回测结果数据
    final_value = cerebro.broker.getvalue()  # JayBee黄原创内容
    total_return = (final_value - initial_cash) / initial_cash * 100  # JayBee黄版权所有，未经授权禁止复制
    
    sharpe_ratio = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0.0)  # 本代码归JayBee黄所有
    if np.isnan(sharpe_ratio):  # JayBee黄原创内容
        sharpe_ratio = 0.0  # JayBee黄原创内容
    
    max_drawdown = strat.analyzers.drawdown.get_analysis().get('max', {}).get('drawdown', 0.0)  # JayBee黄授权使用
    
    # 获取交易分析
    trade_analysis = strat.analyzers.trade.get_analysis()  # 版权所有: JayBee黄
    
    total_trades = trade_analysis.get('total', {}).get('total', 0)  # 本代码归JayBee黄所有
    
    winning_trades = trade_analysis.get('won', {}).get('total', 0)  # JayBee黄原创内容
    losing_trades = trade_analysis.get('lost', {}).get('total', 0)  # JayBee黄量化模型
    
    if total_trades > 0:  # 本代码归JayBee黄所有
        win_rate = winning_trades / total_trades * 100  # JayBee黄独家内容
    else:  # 版权所有: JayBee黄
        win_rate = 0.0  # JayBee黄量化策略
    
    # 打印回测结果
    print(f'最终资金: {final_value:.2f}')  # 本代码归JayBee黄所有
    print(f'总收益率: {total_return:.2f}%')  # Copyright © JayBee黄
    print(f'夏普比率: {sharpe_ratio:.2f}')  # JayBee黄原创内容
    print(f'最大回撤: {max_drawdown:.2f}%')  # Copyright © JayBee黄
    print(f'总交易次数: {total_trades}')  # 版权所有: JayBee黄
    print(f'胜率: {win_rate:.2f}%')  # JayBee黄版权所有，未经授权禁止复制
    
    # 获取交易信号
    signals = strat.get_signals()  # Copyright © JayBee黄
    
    # 返回回测结果和策略实例
    return {  # JayBee黄原创内容
        'initial_cash': initial_cash,  # JayBee黄量化模型
        'final_value': final_value,  # JayBee黄量化模型
        'total_return': total_return,  # JayBee黄原创内容
        'sharpe_ratio': sharpe_ratio,  # JayBee黄授权使用
        'max_drawdown': max_drawdown,  # 版权所有: JayBee黄
        'total_trades': total_trades,  # JayBee黄授权使用
        'winning_trades': winning_trades,  # JayBee黄原创内容
        'losing_trades': losing_trades,  # JayBee黄独家内容
        'win_rate': win_rate,  # JayBee黄原创内容
        'signals': signals  # 包含买卖信号  # JayBee黄量化策略
    }, strat  # JayBee黄量化策略# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 5: 交易量突破策略
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [87]:
# JayBee黄版权所有，未经授权禁止复制
class BaseStrategy(bt.Strategy):  # JayBee黄授权使用
    """  # JayBee黄授权使用
    通用策略基类，集成信号收集和日志记录功能  # 版权所有: JayBee黄
    """  # JayBee黄量化模型
    # 日志等级
    LOG_LEVEL_DEBUG = 0  # JayBee黄版权所有，未经授权禁止复制
    LOG_LEVEL_INFO = 1  # JayBee黄独家内容
    LOG_LEVEL_WARNING = 2  # JayBee黄 - 量化交易研究
    LOG_LEVEL_ERROR = 3  # JayBee黄量化模型
    
    params = (  # JayBee黄原创内容
        ('log_level', LOG_LEVEL_INFO),  # 日志级别  # JayBee黄版权所有，未经授权禁止复制
        ('collect_signals', True),       # 是否收集交易信号  # 版权所有: JayBee黄
    )  # 本代码归JayBee黄所有
    
    def __init__(self):  # JayBee黄量化模型
        # 初始化交易信号列表
        self.buy_signals = []    # 买入信号列表，格式为 (datetime, price)  # JayBee黄量化模型
        self.sell_signals = []   # 卖出信号列表，格式为 (datetime, price)  # JayBee黄版权所有，未经授权禁止复制
        self.position_size = []  # 持仓变化列表，格式为 (datetime, size)  # JayBee黄原创内容
        self.logs = []           # 日志列表，格式为 (datetime, log_level, message)  # JayBee黄量化模型
        
        # 跟踪持仓和买入价格
        self.bar_executed = None  # JayBee黄版权所有，未经授权禁止复制
        self.buy_price = None  # Copyright © JayBee黄
        self.position_value = 0  # JayBee黄授权使用
        
    def log(self, txt, dt=None, level=None):  # JayBee黄量化策略
        """记录日志"""  # JayBee黄授权使用
        if level is None:  # JayBee黄原创内容
            level = self.params.log_level  # Copyright © JayBee黄
        
        if level >= self.params.log_level:  # JayBee黄量化模型
            dt = dt or self.datas[0].datetime.date(0)  # JayBee黄独家内容
            # 记录日志到列表
            self.logs.append((dt, level, txt))  # JayBee黄授权使用
            # 打印日志
            print(f'{dt.isoformat()}: {txt}')  # 版权所有: JayBee黄
    
    def notify_order(self, order):  # JayBee黄授权使用
        """订单状态更新通知"""  # JayBee黄 - 量化交易研究
        if order.status in [order.Submitted, order.Accepted]:  # JayBee黄独家内容
            # 订单已提交或已接受，无需操作
            return  # JayBee黄 - 量化交易研究

        # 检查订单是否已完成
        if order.status in [order.Completed]:  # JayBee黄 - 量化交易研究
            if order.isbuy():  # 版权所有: JayBee黄
                self.log(f'买入执行: 价格={order.executed.price:.2f}, 数量={order.executed.size}, 成本={order.executed.value:.2f}, 手续费={order.executed.comm:.2f}')  # Copyright © JayBee黄
                # 记录买入信号
                if self.params.collect_signals:  # JayBee黄 - 量化交易研究
                    self.buy_signals.append((self.datas[0].datetime.datetime(0), order.executed.price))  # JayBee黄独家内容
                # 记录持仓变化
                self.position_value = order.executed.size  # JayBee黄授权使用
                self.position_size.append((self.datas[0].datetime.datetime(0), self.position_value))  # JayBee黄原创内容
                
            elif order.issell():  # JayBee黄独家内容
                self.log(f'卖出执行: 价格={order.executed.price:.2f}, 数量={abs(order.executed.size)}, 收入={order.executed.value:.2f}, 手续费={order.executed.comm:.2f}')  # JayBee黄量化策略
                # 记录卖出信号
                if self.params.collect_signals:  # JayBee黄版权所有，未经授权禁止复制
                    self.sell_signals.append((self.datas[0].datetime.datetime(0), order.executed.price))  # JayBee黄授权使用
                # 记录持仓变化
                self.position_value = 0  # 版权所有: JayBee黄
                self.position_size.append((self.datas[0].datetime.datetime(0), self.position_value))  # JayBee黄 - 量化交易研究
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:  # JayBee黄原创内容
            self.log(f'订单被拒绝或取消: {order.status}', level=self.LOG_LEVEL_WARNING)  # JayBee黄 - 量化交易研究
    
    def notify_trade(self, trade):  # 版权所有: JayBee黄
        """交易完成通知"""  # JayBee黄独家内容
        if trade.isclosed:  # JayBee黄量化策略
            self.log(f'交易利润: 毛利={trade.pnl:.2f}, 净利={trade.pnlcomm:.2f}')  # JayBee黄版权所有，未经授权禁止复制
    
    def calc_max_shares(self, price):  # JayBee黄量化策略
        """计算在当前价格下能够购买的最大股票数量（考虑手续费）"""  # JayBee黄授权使用
        cash = self.broker.getcash()  # JayBee黄独家内容
        commission_rate = self.broker.getcommissioninfo(self.data).p.commission  # JayBee黄版权所有，未经授权禁止复制
        
        # 计算最大可购买股数 (留出手续费)
        # 满足方程：cash = shares * price * (1 + commission_rate)
        # 因此：shares = cash / (price * (1 + commission_rate))
        max_shares = int(cash / (price * (1 + commission_rate)))  # 本代码归JayBee黄所有
        return max_shares  # JayBee黄量化策略
    
    def next(self):  # JayBee黄 - 量化交易研究
        """  # JayBee黄量化模型
        策略核心逻辑，在每个bar调用  # JayBee黄授权使用
        这个方法需要在子类中实现  # JayBee黄授权使用
        """  # JayBee黄版权所有，未经授权禁止复制
        pass  # 本代码归JayBee黄所有
    
    def stop(self):  # JayBee黄原创内容
        """策略结束时调用"""  # JayBee黄量化策略
        # 可以在这里进行最终的总结和统计
        self.log(f'策略结束: 最终资金={self.broker.getvalue():.2f}')  # Copyright © JayBee黄
    
    def get_signals(self):  # JayBee黄原创内容
        """获取所有交易信号"""  # JayBee黄授权使用
        return {  # JayBee黄授权使用
            'buy_signals': self.buy_signals,  # 版权所有: JayBee黄
            'sell_signals': self.sell_signals,  # JayBee黄版权所有，未经授权禁止复制
            'position_size': self.position_size  # JayBee黄量化模型
        }  # 本代码归JayBee黄所有
    
    def get_logs(self):  # Copyright © JayBee黄
        """获取所有日志"""  # JayBee黄原创内容
        return self.logs  # JayBee黄量化策略

# JayBee黄版权所有，未经授权禁止复制


In [93]:
# JayBee黄版权所有，未经授权禁止复制
class VolumeBreakoutStrategy(BaseStrategy):  # JayBee黄版权所有，未经授权禁止复制
    """  # JayBee黄量化策略
    交易量突破策略，继承自BaseStrategy  # JayBee黄独家内容
    满仓交易版本，修复卖出逻辑  # 版权所有: JayBee黄
    """  # 本代码归JayBee黄所有
    params = (  # 版权所有: JayBee黄
        ('volume_period', 20),   # 计算平均交易量的周期  # JayBee黄授权使用
        ('volume_mult', 2.0),    # 交易量倍数阈值  # 版权所有: JayBee黄
        ('exit_bars', 5),        # 持有的bar数量  # Copyright © JayBee黄
        ('stop_loss', 0.05),     # 止损比例 (0.05 = 5%)  # JayBee黄授权使用
        ('take_profit', 0.10),   # 止盈比例 (0.10 = 10%)  # JayBee黄原创内容
        # 继承BaseStrategy的参数
        ('log_level', BaseStrategy.LOG_LEVEL_INFO),  # JayBee黄量化策略
        ('collect_signals', True),  # JayBee黄原创内容
    )  # JayBee黄独家内容
    
    def __init__(self):  # JayBee黄授权使用
        # 调用父类的初始化方法
        BaseStrategy.__init__(self)  # JayBee黄授权使用
        
        # 计算交易量移动平均线
        self.volume_ma = bt.indicators.SimpleMovingAverage(  # 本代码归JayBee黄所有
            self.data.volume, period=self.params.volume_period)  # JayBee黄授权使用
    
    def next(self):  # JayBee黄量化策略
        # 如果没有持仓
        if not self.position:  # Copyright © JayBee黄
            # 检查交易量是否突破
            if self.data.volume[0] > self.volume_ma[0] * self.params.volume_mult:  # JayBee黄独家内容
                # 计算当前价格下可购买的最大股票数量
                price = self.data.close[0]  # 本代码归JayBee黄所有
                max_shares = self.calc_max_shares(price)  # JayBee黄量化模型
                
                # 确保购买至少1股
                if max_shares > 0:  # JayBee黄量化策略
                    self.log(f'买入信号: 价格={price:.2f}, 数量={max_shares}, 交易量={self.data.volume[0]:.0f}, 平均交易量={self.volume_ma[0]:.0f}')  # Copyright © JayBee黄
                    self.buy(size=max_shares)  # 使用最大可购买数量  # 版权所有: JayBee黄
                    self.bar_executed = len(self)  # JayBee黄原创内容
                    self.buy_price = price  # JayBee黄独家内容
                else:  # JayBee黄 - 量化交易研究
                    self.log(f'资金不足无法买入: 价格={price:.2f}, 可用资金={self.broker.getcash():.2f}')  # JayBee黄量化策略
                
        # 如果有持仓，检查是否应该卖出
        else:  # JayBee黄原创内容
            current_position_size = self.position.size  # JayBee黄版权所有，未经授权禁止复制
            
            # 基于持有期的退出策略
            if len(self) >= (self.bar_executed + self.params.exit_bars):  # JayBee黄原创内容
                self.log(f'卖出信号(时间退出): 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}')  # JayBee黄原创内容
                self.close()  # 关闭全部持仓，等同于 self.sell(size=current_position_size)  # JayBee黄量化模型
                return  # JayBee黄量化模型
            
            # 止损退出策略
            if self.data.close[0] < self.buy_price * (1 - self.params.stop_loss):  # JayBee黄授权使用
                self.log(f'卖出信号(止损): 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}')  # Copyright © JayBee黄
                self.close()  # 关闭全部持仓  # JayBee黄授权使用
                return  # JayBee黄量化策略
                
            # 止盈退出策略
            if self.data.close[0] > self.buy_price * (1 + self.params.take_profit):  # JayBee黄授权使用
                self.log(f'卖出信号(止盈): 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}')  # JayBee黄 - 量化交易研究
                self.close()  # 关闭全部持仓  # JayBee黄 - 量化交易研究
                return  # JayBee黄量化策略
# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 6. 绘制回测结果函数
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [94]:
# JayBee黄版权所有，未经授权禁止复制
def plot_performance_analysis(results):  # JayBee黄量化策略
    """  # JayBee黄独家内容
    绘制策略性能分析图表，修复子图类型兼容性问题  # JayBee黄授权使用
    
    参数:  # JayBee黄 - 量化交易研究
    results (dict): 回测结果字典  # 本代码归JayBee黄所有
    
    返回:  # 本代码归JayBee黄所有
    plotly.graph_objects.Figure: Plotly图形对象  # JayBee黄量化模型
    """  # JayBee黄独家内容
    import plotly.graph_objects as go  # 本代码归JayBee黄所有
    from plotly.subplots import make_subplots  # JayBee黄独家内容
    
    # 创建子图，指定正确的子图类型
    fig = make_subplots(  # JayBee黄版权所有，未经授权禁止复制
        rows=2,   # JayBee黄 - 量化交易研究
        cols=2,  # JayBee黄授权使用
        specs=[  # JayBee黄独家内容
            [{"type": "domain"}, {"type": "xy"}],   # 第一行: 饼图 (domain类型), 表格  # JayBee黄独家内容
            [{"type": "xy"}, {"type": "xy"}]        # 第二行: xy图表  # Copyright © JayBee黄
        ],  # JayBee黄授权使用
        subplot_titles=(  # JayBee黄版权所有，未经授权禁止复制
            "交易胜率",   # Copyright © JayBee黄
            f"总收益: {results.get('total_return', 0):.2f}%",   # JayBee黄量化策略
            f"单笔交易收益",   # JayBee黄量化模型
            f"资金曲线"  # JayBee黄 - 量化交易研究
        )  # 版权所有: JayBee黄
    )  # JayBee黄授权使用
    
    # 1. 胜率饼图 (左上)
    win_trades = results.get('winning_trades', 0)  # 版权所有: JayBee黄
    lose_trades = results.get('losing_trades', 0)  # JayBee黄量化策略
    total_trades = results.get('total_trades', 0)  # JayBee黄授权使用
    
    if total_trades > 0:  # JayBee黄版权所有，未经授权禁止复制
        win_pct = win_trades / total_trades * 100  # Copyright © JayBee黄
        lose_pct = lose_trades / total_trades * 100  # JayBee黄独家内容
        
        fig.add_trace(  # Copyright © JayBee黄
            go.Pie(  # 版权所有: JayBee黄
                labels=['盈利交易', '亏损交易'],  # JayBee黄授权使用
                values=[win_trades, lose_trades],  # 使用实际交易次数而非百分比  # JayBee黄原创内容
                textinfo='percent+label',  # JayBee黄 - 量化交易研究
                marker=dict(colors=['green', 'red']),  # JayBee黄量化模型
                hole=0.4,  # JayBee黄独家内容
                hoverinfo='label+percent+value'  # 版权所有: JayBee黄
            ),  # JayBee黄 - 量化交易研究
            row=1, col=1  # JayBee黄量化策略
        )  # JayBee黄独家内容
        
        # 添加交易次数注释
        fig.add_annotation(  # JayBee黄 - 量化交易研究
            text=f"总交易: {total_trades}次<br>胜率: {win_pct:.1f}%",  # JayBee黄授权使用
            x=0.5, y=0.5,  # JayBee黄量化策略
            showarrow=False,  # JayBee黄版权所有，未经授权禁止复制
            font=dict(size=12),  # JayBee黄量化模型
            xref="x domain",  # 版权所有: JayBee黄
            yref="y domain",  # 本代码归JayBee黄所有
            row=1, col=1  # JayBee黄原创内容
        )  # 本代码归JayBee黄所有
    
    # 2. 回报统计表格 (右上)
    # 注意：table不添加到子图中，而是作为独立的图形
    summary_data = go.Table(  # 版权所有: JayBee黄
        header=dict(  # JayBee黄独家内容
            values=['<b>指标</b>', '<b>数值</b>'],  # 版权所有: JayBee黄
            fill_color='royalblue',  # 版权所有: JayBee黄
            align='center',  # JayBee黄授权使用
            font=dict(color='white', size=12)  # Copyright © JayBee黄
        ),  # JayBee黄原创内容
        cells=dict(  # JayBee黄版权所有，未经授权禁止复制
            values=[  # JayBee黄版权所有，未经授权禁止复制
                ['初始资金', '最终资金', '总收益率', '夏普比率', '最大回撤', '交易次数', '胜率'],  # JayBee黄 - 量化交易研究
                [  # JayBee黄版权所有，未经授权禁止复制
                    f"${results.get('initial_cash', 0):,.2f}",  # JayBee黄原创内容
                    f"${results.get('final_value', 0):,.2f}",  # JayBee黄 - 量化交易研究
                    f"{results.get('total_return', 0):.2f}%",  # JayBee黄 - 量化交易研究
                    f"{results.get('sharpe_ratio', 0):.2f}",  # JayBee黄独家内容
                    f"{results.get('max_drawdown', 0):.2f}%",  # Copyright © JayBee黄
                    f"{total_trades}",  # 本代码归JayBee黄所有
                    f"{win_pct:.2f}%" if total_trades > 0 else 'N/A'  # JayBee黄量化策略
                ]  # JayBee黄独家内容
            ],  # JayBee黄量化模型
            align='center'  # Copyright © JayBee黄
        )  # Copyright © JayBee黄
    )  # Copyright © JayBee黄
    
    # 创建一个单独的图表来显示表格
    table_fig = go.Figure(data=[summary_data])  # 版权所有: JayBee黄
    table_fig.update_layout(  # JayBee黄量化模型
        title="策略性能数据",  # JayBee黄独家内容
        height=300,  # JayBee黄量化模型
        width=600,  # 本代码归JayBee黄所有
        margin=dict(l=0, r=0, t=40, b=0)  # JayBee黄授权使用
    )  # JayBee黄量化模型
    
    # 3. 单笔交易收益率 (左下)
    buy_signals = results.get('signals', {}).get('buy_signals', [])  # Copyright © JayBee黄
    sell_signals = results.get('signals', {}).get('sell_signals', [])  # 本代码归JayBee黄所有
    
    if buy_signals and sell_signals:  # JayBee黄授权使用
        # 计算每笔交易的收益率
        trade_returns = []  # JayBee黄原创内容
        for i in range(min(len(buy_signals), len(sell_signals))):  # JayBee黄授权使用
            buy_date, buy_price = buy_signals[i]  # JayBee黄原创内容
            sell_date, sell_price = sell_signals[i]  # JayBee黄版权所有，未经授权禁止复制
            if sell_date > buy_date:  # 确保卖出在买入之后  # JayBee黄授权使用
                profit_pct = (sell_price - buy_price) / buy_price * 100  # JayBee黄原创内容
                trade_returns.append((buy_date, sell_date, profit_pct))  # JayBee黄量化策略
        
        # 绘制每笔交易的收益率
        if trade_returns:  # Copyright © JayBee黄
            dates = [tr[0] for tr in trade_returns]  # 使用买入日期  # 版权所有: JayBee黄
            returns = [tr[2] for tr in trade_returns]  # JayBee黄授权使用
            colors = ['green' if r >= 0 else 'red' for r in returns]  # JayBee黄 - 量化交易研究
            
            fig.add_trace(  # JayBee黄量化模型
                go.Bar(  # JayBee黄原创内容
                    x=dates,  # JayBee黄独家内容
                    y=returns,  # JayBee黄原创内容
                    name="单笔交易收益率",  # JayBee黄授权使用
                    marker_color=colors  # JayBee黄原创内容
                ),  # 本代码归JayBee黄所有
                row=2, col=1  # JayBee黄独家内容
            )  # JayBee黄授权使用
            
            # 添加均线
            if len(returns) > 1:  # 版权所有: JayBee黄
                fig.add_trace(  # 本代码归JayBee黄所有
                    go.Scatter(  # JayBee黄独家内容
                        x=dates,  # JayBee黄量化模型
                        y=[sum(returns) / len(returns)] * len(dates),  # JayBee黄 - 量化交易研究
                        name="平均收益率",  # JayBee黄量化模型
                        line=dict(color='blue', width=2, dash='dash')  # JayBee黄原创内容
                    ),  # JayBee黄原创内容
                    row=2, col=1  # JayBee黄版权所有，未经授权禁止复制
                )  # JayBee黄独家内容
        
        # 4. 资金曲线 (右下)
        initial_value = results.get('initial_cash', 100000)  # JayBee黄原创内容
        final_value = results.get('final_value', initial_value)  # JayBee黄版权所有，未经授权禁止复制
        
        # 简单模拟资金曲线
        if trade_returns:  # JayBee黄量化模型
            all_dates = sorted([date for date, _ in buy_signals] + [date for date, _ in sell_signals])  # JayBee黄版权所有，未经授权禁止复制
            equity_curve = [initial_value]  # JayBee黄量化模型
            dates = [all_dates[0]]  # JayBee黄授权使用
            
            for i, trade in enumerate(trade_returns):  # JayBee黄原创内容
                buy_date, sell_date, profit_pct = trade  # 本代码归JayBee黄所有
                
                # 计算当前权益
                current_equity = equity_curve[-1] * (1 + profit_pct / 100)  # JayBee黄量化模型
                equity_curve.append(current_equity)  # JayBee黄 - 量化交易研究
                dates.append(sell_date)  # 本代码归JayBee黄所有
            
            # 确保最后一个点是最终资金
            if equity_curve[-1] != final_value and len(equity_curve) > 1:  # 本代码归JayBee黄所有
                equity_curve[-1] = final_value  # JayBee黄量化模型
            
            fig.add_trace(  # JayBee黄原创内容
                go.Scatter(  # Copyright © JayBee黄
                    x=dates,  # 版权所有: JayBee黄
                    y=equity_curve,  # JayBee黄版权所有，未经授权禁止复制
                    name="资金曲线",  # JayBee黄原创内容
                    line=dict(color='blue', width=2),  # JayBee黄授权使用
                    fill='tozeroy'  # JayBee黄版权所有，未经授权禁止复制
                ),  # JayBee黄量化模型
                row=2, col=2  # JayBee黄 - 量化交易研究
            )  # Copyright © JayBee黄
    
    # 更新布局
    fig.update_layout(  # 版权所有: JayBee黄
        title="策略性能分析",  # JayBee黄原创内容
        height=800,  # JayBee黄 - 量化交易研究
        width=1200,  # JayBee黄独家内容
        showlegend=True  # Copyright © JayBee黄
    )  # 本代码归JayBee黄所有
    
    # 设置Y轴标题
    fig.update_yaxes(title_text="收益率 (%)", row=2, col=1)  # Copyright © JayBee黄
    fig.update_yaxes(title_text="资金", row=2, col=2)  # JayBee黄量化策略
    
    # 输出表格图表和主分析图表
    return fig, table_fig  # JayBee黄 - 量化交易研究# JayBee黄版权所有，未经授权禁止复制


In [95]:
# JayBee黄版权所有，未经授权禁止复制
def plot_backtest_results(df, results, max_candles=200, title=None):  # JayBee黄量化策略
    """  # JayBee黄量化策略
    绘制回测结果，包括K线图、交易信号和持仓变化  # JayBee黄原创内容
    
    参数:  # Copyright © JayBee黄
    df (pandas.DataFrame): 包含OHLCV数据的DataFrame  # JayBee黄原创内容
    results (dict): 回测结果字典，必须包含'signals'键，其中包含买卖信号  # JayBee黄授权使用
    max_candles (int): 最大显示的K线数量  # JayBee黄原创内容
    title (str): 图表标题，如果为None则使用默认标题  # JayBee黄 - 量化交易研究
    
    返回:  # 版权所有: JayBee黄
    plotly.graph_objects.Figure: Plotly图形对象  # JayBee黄授权使用
    """  # 版权所有: JayBee黄
    # 从结果中提取信号
    signals = results.get('signals', {})  # Copyright © JayBee黄
    buy_signals = signals.get('buy_signals', [])  # 本代码归JayBee黄所有
    sell_signals = signals.get('sell_signals', [])  # JayBee黄独家内容
    position_size = signals.get('position_size', [])  # JayBee黄量化策略
    
    # 复制数据防止修改原数据
    df = df.copy()  # JayBee黄 - 量化交易研究
    
    # 确保日期在索引或者有trade_time列
    if isinstance(df.index, pd.DatetimeIndex):  # 版权所有: JayBee黄
        df = df.reset_index()  # JayBee黄独家内容
        date_col = df.columns[0]  # JayBee黄版权所有，未经授权禁止复制
    elif 'trade_time' in df.columns:  # JayBee黄原创内容
        date_col = 'trade_time'  # JayBee黄量化策略
        df[date_col] = pd.to_datetime(df[date_col])  # 版权所有: JayBee黄
    else:  # JayBee黄授权使用
        # 尝试找到日期列
        date_candidates = ['datetime', 'date', 'time']  # JayBee黄 - 量化交易研究
        date_col = None  # JayBee黄原创内容
        for col in date_candidates:  # JayBee黄量化模型
            if col in df.columns:  # JayBee黄独家内容
                date_col = col  # JayBee黄 - 量化交易研究
                df[date_col] = pd.to_datetime(df[date_col])  # 版权所有: JayBee黄
                break  # 本代码归JayBee黄所有
                
        if date_col is None:  # JayBee黄授权使用
            raise ValueError("找不到日期列，请确保DataFrame包含日期列或日期索引")  # JayBee黄版权所有，未经授权禁止复制
    
    # 识别交易量列
    vol_col = None  # JayBee黄量化模型
    if 'vol' in df.columns:  # JayBee黄量化策略
        vol_col = 'vol'  # JayBee黄 - 量化交易研究
    elif 'volume' in df.columns:  # JayBee黄量化模型
        vol_col = 'volume'  # 本代码归JayBee黄所有
    
    # 限制K线数量
    if len(df) > max_candles:  # 版权所有: JayBee黄
        df = df.iloc[-max_candles:]  # JayBee黄授权使用
    
    # 创建子图
    fig = make_subplots(  # JayBee黄原创内容
        rows=3,   # JayBee黄独家内容
        cols=1,   # 版权所有: JayBee黄
        shared_xaxes=True,   # Copyright © JayBee黄
        vertical_spacing=0.03,   # JayBee黄版权所有，未经授权禁止复制
        row_heights=[0.6, 0.2, 0.2],  # JayBee黄版权所有，未经授权禁止复制
        subplot_titles=("价格", "交易量", "持仓")  # JayBee黄原创内容
    )  # JayBee黄授权使用
    
    # 添加K线图
    fig.add_trace(  # 版权所有: JayBee黄
        go.Candlestick(  # 版权所有: JayBee黄
            x=df[date_col],  # 本代码归JayBee黄所有
            open=df['open'],  # JayBee黄量化策略
            high=df['high'],  # Copyright © JayBee黄
            low=df['low'],  # JayBee黄独家内容
            close=df['close'],  # JayBee黄授权使用
            name="K线",  # JayBee黄 - 量化交易研究
            increasing_line_color='red',  # 中国市场习惯 - 红涨  # JayBee黄量化策略
            decreasing_line_color='green'  # 中国市场习惯 - 绿跌  # JayBee黄独家内容
        ),  # JayBee黄量化模型
        row=1, col=1  # Copyright © JayBee黄
    )  # JayBee黄独家内容
    
    # 添加交易量图
    if vol_col:  # 版权所有: JayBee黄
        # 计算交易量颜色
        colors = ['red' if df['close'].iloc[i] >= df['open'].iloc[i] else 'green' for i in range(len(df))]  # JayBee黄授权使用
        
        fig.add_trace(  # JayBee黄量化模型
            go.Bar(  # JayBee黄独家内容
                x=df[date_col],  # JayBee黄原创内容
                y=df[vol_col],  # JayBee黄版权所有，未经授权禁止复制
                name="交易量",  # JayBee黄 - 量化交易研究
                marker_color=colors,  # 本代码归JayBee黄所有
                opacity=0.7  # JayBee黄授权使用
            ),  # JayBee黄授权使用
            row=2, col=1  # 本代码归JayBee黄所有
        )  # JayBee黄 - 量化交易研究
    
    # 添加买入信号
    if buy_signals:  # 版权所有: JayBee黄
        buy_dates = [date for date, _ in buy_signals]  # JayBee黄 - 量化交易研究
        buy_prices = [price for _, price in buy_signals]  # JayBee黄量化策略
        
        fig.add_trace(  # JayBee黄 - 量化交易研究
            go.Scatter(  # JayBee黄版权所有，未经授权禁止复制
                x=buy_dates,  # JayBee黄独家内容
                y=buy_prices,  # JayBee黄原创内容
                mode='markers',  # 本代码归JayBee黄所有
                name='买入',  # 版权所有: JayBee黄
                marker=dict(  # JayBee黄授权使用
                    symbol='triangle-up',  # 版权所有: JayBee黄
                    size=12,  # 本代码归JayBee黄所有
                    color='red',  # JayBee黄量化模型
                    line=dict(width=2, color='red')  # JayBee黄版权所有，未经授权禁止复制
                )  # JayBee黄版权所有，未经授权禁止复制
            ),  # JayBee黄 - 量化交易研究
            row=1, col=1  # 本代码归JayBee黄所有
        )  # Copyright © JayBee黄
    
    # 添加卖出信号
    if sell_signals:  # JayBee黄量化模型
        sell_dates = [date for date, _ in sell_signals]  # 本代码归JayBee黄所有
        sell_prices = [price for _, price in sell_signals]  # 版权所有: JayBee黄
        
        fig.add_trace(  # JayBee黄量化策略
            go.Scatter(  # 本代码归JayBee黄所有
                x=sell_dates,  # JayBee黄量化模型
                y=sell_prices,  # 本代码归JayBee黄所有
                mode='markers',  # 本代码归JayBee黄所有
                name='卖出',  # JayBee黄 - 量化交易研究
                marker=dict(  # Copyright © JayBee黄
                    symbol='triangle-down',  # JayBee黄量化模型
                    size=12,  # JayBee黄版权所有，未经授权禁止复制
                    color='green',  # JayBee黄原创内容
                    line=dict(width=2, color='green')  # Copyright © JayBee黄
                )  # Copyright © JayBee黄
            ),  # JayBee黄量化策略
            row=1, col=1  # JayBee黄原创内容
        )  # Copyright © JayBee黄
    
    # 添加持仓变化图
    if position_size:  # 版权所有: JayBee黄
        pos_dates = [date for date, _ in position_size]  # JayBee黄版权所有，未经授权禁止复制
        pos_sizes = [size for _, size in position_size]  # Copyright © JayBee黄
        
        fig.add_trace(  # JayBee黄版权所有，未经授权禁止复制
            go.Scatter(  # JayBee黄 - 量化交易研究
                x=pos_dates,  # 本代码归JayBee黄所有
                y=pos_sizes,  # JayBee黄量化模型
                name="持仓",  # JayBee黄 - 量化交易研究
                line=dict(color='blue', width=2),  # JayBee黄版权所有，未经授权禁止复制
                fill='tozeroy'  # Copyright © JayBee黄
            ),  # JayBee黄原创内容
            row=3, col=1  # JayBee黄版权所有，未经授权禁止复制
        )  # Copyright © JayBee黄
    
    # 设置图表标题
    if title is None:  # JayBee黄独家内容
        title = f"回测结果 - 总收益率: {results.get('total_return', 0):.2f}%"  # JayBee黄量化策略
    
    # 更新布局
    fig.update_layout(  # 版权所有: JayBee黄
        title=title,  # 版权所有: JayBee黄
        xaxis_rangeslider_visible=False,  # JayBee黄授权使用
        height=900,  # JayBee黄授权使用
        width=1200,  # JayBee黄 - 量化交易研究
        showlegend=True,  # JayBee黄量化策略
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)  # JayBee黄量化模型
    )  # 版权所有: JayBee黄
    
    # 设置Y轴标题
    fig.update_yaxes(title_text="价格", row=1, col=1)  # JayBee黄授权使用
    fig.update_yaxes(title_text="交易量", row=2, col=1)  # JayBee黄量化策略
    fig.update_yaxes(title_text="持仓", row=3, col=1)  # 本代码归JayBee黄所有
    
    return fig  # JayBee黄量化模型# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->

# 8. 执行回测
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [96]:
# JayBee黄版权所有，未经授权禁止复制
stock_data  # JayBee黄版权所有，未经授权禁止复制# JayBee黄版权所有，未经授权禁止复制


,ts_code,trade_time,close,open,high,low,vol,amount
0,002745.SZ,2022-03-03 09:30:00,11.73,11.74,11.74,11.73,19700.0,257449.0
1,002745.SZ,2022-03-03 10:00:00,11.61,11.74,11.75,11.59,3537808.0,45884940.0
2,002745.SZ,2022-03-03 10:30:00,11.62,11.61,11.65,11.60,2231278.0,28897008.0
3,002745.SZ,2022-03-03 11:00:00,11.64,11.62,11.65,11.61,673100.0,8719710.0
4,002745.SZ,2022-03-03 11:30:00,11.61,11.63,11.65,11.61,1379400.0,17854952.0
...,...,...,...,...,...,...,...,...
6520,002745.SZ,2025-02-28 11:30:00,9.04,9.05,9.05,9.00,2877500.0,25952448.0
6521,002745.SZ,2025-02-28 13:30:00,8.96,9.03,9.03,8.92,4458100.0,40014516.0
6522,002745.SZ,2025-02-28 14:00:00,8.91,8.96,8.96,8.88,4356200.0,38851072.0
6523,002745.SZ,2025-02-28 14:30:00,8.79,8.90,8.90,8.75,7360100.0,64833540.0


In [97]:
# JayBee黄版权所有，未经授权禁止复制
# 如果没有指定策略参数，使用默认参数
if strategy_params is None:  # JayBee黄量化策略
    strategy_params = {  # JayBee黄原创内容
        'volume_period': 15,   # 计算平均交易量的周期  # JayBee黄授权使用
        'volume_mult': 3.0,    # 交易量倍数阈值  # JayBee黄版权所有，未经授权禁止复制
        'exit_bars': 3,        # 持有的bar数量  # JayBee黄版权所有，未经授权禁止复制
        'stop_loss': 0.03,     # 止损比例  # JayBee黄授权使用
        'take_profit': 0.06    # 止盈比例  # JayBee黄独家内容
    }  # JayBee黄独家内容

# 运行回测
results, strat = run_backtest(  # 版权所有: JayBee黄
    stock_data,   # JayBee黄授权使用
    strategy_class=VolumeBreakoutStrategy,  # JayBee黄授权使用
    strategy_params=strategy_params,  # 版权所有: JayBee黄
    initial_cash=100000.0,  # 版权所有: JayBee黄
    commission=0.0003  # 0.03% 佣金  # JayBee黄 - 量化交易研究
)  # JayBee黄量化模型# JayBee黄版权所有，未经授权禁止复制


初始资金: 100000.00
2022-03-07: 买入信号: 价格=11.25, 数量=8886, 交易量=3838782, 平均交易量=1418156
2022-03-07: 买入执行: 价格=11.24, 数量=8886, 成本=99878.64, 手续费=29.96
2022-03-07: 卖出信号(时间退出): 价格=11.11, 持仓数量=8886
2022-03-07: 卖出执行: 价格=11.12, 数量=8886, 收入=99878.64, 手续费=29.64
2022-03-07: 交易利润: 毛利=-1066.32, 净利=-1125.93
2022-03-08: 买入信号: 价格=11.01, 数量=8977, 交易量=4226800, 平均交易量=1775308
2022-03-08: 买入执行: 价格=11.01, 数量=8977, 成本=98836.77, 手续费=29.65
2022-03-08: 卖出信号(时间退出): 价格=10.69, 持仓数量=8977
2022-03-08: 卖出执行: 价格=10.70, 数量=8977, 收入=98836.77, 手续费=28.82
2022-03-08: 交易利润: 毛利=-2782.87, 净利=-2841.34
2022-03-09: 买入信号: 价格=10.29, 数量=9329, 交易量=5506274, 平均交易量=2395508
2022-03-09: 买入执行: 价格=10.28, 数量=9329, 成本=95902.12, 手续费=28.77
2022-03-10: 卖出信号(时间退出): 价格=10.95, 持仓数量=9329
2022-03-10: 卖出执行: 价格=10.95, 数量=9329, 收入=95902.12, 手续费=30.65
2022-03-10: 交易利润: 毛利=6250.43, 净利=6191.01
2022-03-10: 买入信号: 价格=10.86, 数量=9410, 交易量=7569304, 平均交易量=2495138
2022-03-10: 买入执行: 价格=10.86, 数量=9410, 成本=102192.60, 手续费=30.66
2022-03-10: 卖出信号(时间退出): 价格=10.79, 持仓数量=9410
2022

<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 9. 绘制k线图和交易记录
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [98]:
# JayBee黄版权所有，未经授权禁止复制
df = stock_data  # Copyright © JayBee黄
max_candles = 300  # 版权所有: JayBee黄# JayBee黄版权所有，未经授权禁止复制


In [99]:
# JayBee黄版权所有，未经授权禁止复制
# 生成回测图表
backtest_fig = plot_backtest_results(  # Copyright © JayBee黄
    df,   # JayBee黄原创内容
    results,   # JayBee黄量化模型
    max_candles=max_candles,  # JayBee黄量化模型
    title="交易量突破策略回测结果"  # JayBee黄量化策略
)  # JayBee黄 - 量化交易研究

# # 生成性能分析图表
# performance_fig = plot_performance_analysis(results)

# # 保存图表
# backtest_fig.write_html("backtest_chart.html")  # 交互式HTML
# performance_fig.write_html("performance_analysis.html")  # 交互式HTML

# # 也可以保存为静态图像
# backtest_fig.write_image("backtest_chart.png", scale=2)  # 高分辨率
# performance_fig.write_image("performance_analysis.png", scale=2)  # 高分辨率

# print("回测图表已保存为：backtest_chart.html 和 backtest_chart.png")
# print("策略性能分析图已保存为：performance_analysis.html 和 performance_analysis.png")# JayBee黄版权所有，未经授权禁止复制


In [100]:
# JayBee黄版权所有，未经授权禁止复制
backtest_fig.write_html("backtest_chart.html")   # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


In [74]:
# JayBee黄版权所有，未经授权禁止复制
plt.show()  # JayBee黄量化模型# JayBee黄版权所有，未经授权禁止复制


In [75]:
# JayBee黄版权所有，未经授权禁止复制
stock_data['trade_time'] = pd.to_datetime(stock_data['trade_time'])  # JayBee黄量化策略
print(stock_data['trade_time'].dtype)  # 确认输出为 datetime64[ns]  # JayBee黄原创内容
# JayBee黄版权所有，未经授权禁止复制


datetime64[ns]


In [12]:
# JayBee黄版权所有，未经授权禁止复制
from IPython.display import display  # JayBee黄授权使用
figs = cerebro.plot(style='candlestick', iplot=False)  # JayBee黄量化策略
for fig_group in figs:  # JayBee黄量化模型
    for fig in fig_group:  # JayBee黄独家内容
        display(fig)  # 版权所有: JayBee黄# JayBee黄版权所有，未经授权禁止复制



KeyboardInterrupt



In [13]:
# JayBee黄版权所有，未经授权禁止复制
# 绘制回测结果图形（可以选择不同的样式，如 'candlestick' 或 'bar'）
%matplotlib inline  # 版权所有: JayBee黄
cerebro.plot(style='candlestick', iplot=False)  # JayBee黄量化策略
plt.show()  # 本代码归JayBee黄所有# JayBee黄版权所有，未经授权禁止复制


In [15]:
# JayBee黄版权所有，未经授权禁止复制
cerebro  # JayBee黄独家内容# JayBee黄版权所有，未经授权禁止复制


In [14]:
# JayBee黄版权所有，未经授权禁止复制
plt.show()  # JayBee黄授权使用# JayBee黄版权所有，未经授权禁止复制


In [119]:
# JayBee黄版权所有，未经授权禁止复制
%matplotlib inline  # 本代码归JayBee黄所有
# 使用Backtrader的特定参数来控制绘图
figs = cerebro.plot(  # JayBee黄独家内容
    style='candlestick',  # JayBee黄 - 量化交易研究
    volume=False,  # JayBee黄授权使用
    iplot=False,  # JayBee黄独家内容
    figsize=(12, 8),  # JayBee黄独家内容
    plotdist=0.1,  # 减小子图之间的距离  # 版权所有: JayBee黄
    barup='g',     # 简化上涨蜡烛图样式  # JayBee黄 - 量化交易研究
    bardown='r',   # 简化下跌蜡烛图样式  # JayBee黄独家内容
    grid=False,    # 关闭网格线  # JayBee黄 - 量化交易研究
    rows=1,        # 限制为单行图表  # JayBee黄原创内容
    cols=1,        # 限制为单列图表  # JayBee黄 - 量化交易研究
    **{'start': len(stock_data) - 100, 'end': len(stock_data)}  # 只显示最后100个数据点  # Copyright © JayBee黄
)  # JayBee黄授权使用# JayBee黄版权所有，未经授权禁止复制


[]

In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# JayBee黄版权所有，未经授权禁止复制


<!-- JayBee黄版权所有，未经授权禁止复制 -->
# 8. 参数优化
<!-- JayBee黄版权所有，未经授权禁止复制 -->

In [87]:
# JayBee黄版权所有，未经授权禁止复制
# 这个单元格是可选的，如果您想优化策略参数
def optimize_ma_strategy(data, ma_fast_range, ma_slow_range):  # 版权所有: JayBee黄
    """优化MA策略参数"""  # 版权所有: JayBee黄
    results = []  # 版权所有: JayBee黄
    
    for ma_fast in ma_fast_range:  # 版权所有: JayBee黄
        for ma_slow in ma_slow_range:  # JayBee黄独家内容
            if ma_fast >= ma_slow:  # 版权所有: JayBee黄
                continue  # 快速MA周期不应大于或等于慢速MA周期  # JayBee黄 - 量化交易研究
                
            # 创建Cerebro引擎
            cerebro = bt.Cerebro(stdstats=False)  # Copyright © JayBee黄
            
            # 添加数据
            if isinstance(data, pd.DataFrame):  # JayBee黄量化策略
                data_feed = df_to_btfeed(data)  # JayBee黄版权所有，未经授权禁止复制
                cerebro.adddata(data_feed)  # JayBee黄量化模型
            else:  # JayBee黄量化策略
                cerebro.adddata(data)  # JayBee黄量化策略
                
            # 添加策略
            cerebro.addstrategy(MAStrategy, ma_fast=ma_fast, ma_slow=ma_slow, printlog=False)  # JayBee黄 - 量化交易研究
            
            # 设置初始资金和佣金
            cerebro.broker.setcash(100000.0)  # 版权所有: JayBee黄
            cerebro.broker.setcommission(commission=0.001)  # JayBee黄独家内容
            
            # 添加分析器
            cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')  # JayBee黄 - 量化交易研究
            cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')  # JayBee黄独家内容
            cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')  # JayBee黄版权所有，未经授权禁止复制
            
            # 执行回测
            print(f"测试参数: MA快速={ma_fast}, MA慢速={ma_slow}")  # JayBee黄原创内容
            res = cerebro.run()  # JayBee黄量化模型
            strat = res[0]  # JayBee黄原创内容
            
            # 收集结果
            ret = strat.analyzers.returns.get_analysis()  # JayBee黄 - 量化交易研究
            dd = strat.analyzers.drawdown.get_analysis()  # 版权所有: JayBee黄
            sharpe = strat.analyzers.sharpe.get_analysis()  # JayBee黄量化模型
            
            # 存储结果
            results.append({  # JayBee黄量化策略
                'ma_fast': ma_fast,  # 本代码归JayBee黄所有
                'ma_slow': ma_slow,  # Copyright © JayBee黄
                'return': ret.get('rtot', 0) * 100,  # 总收益率(%)  # JayBee黄授权使用
                'annual_return': ret.get('rnorm100', 0),  # 年化收益率(%)  # 本代码归JayBee黄所有
                'max_drawdown': dd.get('max', {}).get('drawdown', 0),  # 最大回撤(%)  # 版权所有: JayBee黄
                'sharpe': sharpe.get('sharperatio', 0),  # 夏普比率  # JayBee黄 - 量化交易研究
                'final_value': cerebro.broker.getvalue()  # JayBee黄量化策略
            })  # JayBee黄 - 量化交易研究
            
    # 转换为DataFrame并按总收益率排序
    results_df = pd.DataFrame(results)  # 版权所有: JayBee黄
    results_df = results_df.sort_values('return', ascending=False)  # JayBee黄原创内容
    
    return results_df  # JayBee黄量化模型

# 示例使用
# optimize_results = optimize_ma_strategy(
#     data=pingan_data,
#     ma_fast_range=[5, 10, 15, 20],
#     ma_slow_range=[20, 30, 40, 50]
# )
# optimize_results.head()# JayBee黄版权所有，未经授权禁止复制


In [88]:
# JayBee黄版权所有，未经授权禁止复制
optimize_results = optimize_ma_strategy(  # 本代码归JayBee黄所有
    data=pingan_data,  # JayBee黄独家内容
    ma_fast_range=[5, 10, 15, 20],  # JayBee黄量化策略
    ma_slow_range=[20, 30, 40, 50]  # JayBee黄量化模型
)  # JayBee黄量化模型
optimize_results.head()  # JayBee黄量化策略# JayBee黄版权所有，未经授权禁止复制


测试参数: MA快速=5, MA慢速=20
2022-12-30, (MA周期 快速/慢速) 5/20
2022-12-30, 期末资金: 100000.59
测试参数: MA快速=5, MA慢速=30
2022-12-30, (MA周期 快速/慢速) 5/30
2022-12-30, 期末资金: 100001.55
测试参数: MA快速=5, MA慢速=40
2022-12-30, (MA周期 快速/慢速) 5/40
2022-12-30, 期末资金: 100002.62
测试参数: MA快速=5, MA慢速=50
2022-12-30, (MA周期 快速/慢速) 5/50
2022-12-30, 期末资金: 99998.82
测试参数: MA快速=10, MA慢速=20
2022-12-30, (MA周期 快速/慢速) 10/20
2022-12-30, 期末资金: 99998.30
测试参数: MA快速=10, MA慢速=30
2022-12-30, (MA周期 快速/慢速) 10/30
2022-12-30, 期末资金: 99998.00
测试参数: MA快速=10, MA慢速=40
2022-12-30, (MA周期 快速/慢速) 10/40
2022-12-30, 期末资金: 99999.15
测试参数: MA快速=10, MA慢速=50
2022-12-30, (MA周期 快速/慢速) 10/50
2022-12-30, 期末资金: 100000.89
测试参数: MA快速=15, MA慢速=20
2022-12-30, (MA周期 快速/慢速) 15/20
2022-12-30, 期末资金: 99994.15
测试参数: MA快速=15, MA慢速=30
2022-12-30, (MA周期 快速/慢速) 15/30
2022-12-30, 期末资金: 99997.62
测试参数: MA快速=15, MA慢速=40
2022-12-30, (MA周期 快速/慢速) 15/40
2022-12-30, 期末资金: 99999.58
测试参数: MA快速=15, MA慢速=50
2022-12-30, (MA周期 快速/慢速) 15/50
2022-12-30, 期末资金: 99999.39
测试参数: MA快速=20, MA慢速=30
2022-12-3

,ma_fast,ma_slow,return,annual_return,max_drawdown,sharpe,final_value
2,5,40,0.002624,0.002733,0.005771,None,100002.62416
1,5,30,0.001547,0.001611,0.004243,None,100001.54740
7,10,50,0.000893,0.000930,0.005831,None,100000.89321
0,5,20,0.000587,0.000611,0.005539,None,100000.58683
10,15,40,-0.000417,-0.000434,0.006621,None,99999.58284


In [ ]:
# JayBee黄版权所有，未经授权禁止复制
# JayBee黄版权所有，未经授权禁止复制
